# Langchain 시작하기

## 설치

Langchain 을 Python 환경에서 사용하기 위해서는, 다음의 명령어를 통하여 설치가 가능하다.

> `nix` 및 `poetry` 를 통해서 환경설정을 한 경우에는 다음의 설치가 필요치 않아서,
> 필요한 경우에는 주석처리를 풀고 실행하면 된다.

In [1]:
# ! pip install langchain

## 환경설정

Langchain 은 주로 OpenAI 의 API 를 통해서 Access 를 함으로, OpenAI 패키지를 설치해야 한다.

> `nix` 및 `poetry` 를 통해서 환경설정을 한 경우에는 다음의 설치가 필요치 않아서,
> 필요한 경우에는 주석처리를 풀고 실행하면 된다.

In [2]:
# ! pip install openai

그리고, OpenAI 에 Access 하기 위해서 사용할 API Key 를 환경변수로 설정해주어야 한다.

환경변수 이름은 `OPENAI_API_KEY` 이며, 다음의 두가지 방식으로 설정 가능하다.

In [1]:
# ! export OPENAI_API_KEY="..."

Python (혹은 `Jupyter Notebook`) 환경에서는, 다음의 코드를 통해서 `OPENAI_API_KEY` 를 설정할 수 있다.

In [2]:
# import os
# os.environ["OPENAI_API_KEY"] = "..."

하지만, 코드 (혹은 `Jupyter Notebook`) 에 개인 API Key 가 들어가지 않아야 하기 때문에,
다음의 방법을 통해서 `OPENAI_API_KEY` 를 설정할 것이다.

출처: Keeping credentials safe in Jupyter Notebooks, https://towardsdatascience.com/keeping-credentials-safe-in-jupyter-notebooks-fbd215a8e311

In [4]:
import os
import getpass

In [5]:
OPENAI_API_KEY = getpass.getpass()

········


In [6]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Language Model Application 만들어보기

Langchain 도 설치했고, OpenAI 를 접근하기 위한 환경설정도 모두 끝냈으니, 한번 Langchain 을 통한 LLM 애플리케이션 만들기 맛보기를 해보자.

Langchain 은 여러가지 모듈 (`Module`) 을 제공하고 있어, 모듈을 조합하여 사용하는 형태로 복잡한 Application 을 손쉽게 만들 수 있다.

### LLM 에 예측 (Prediction) 을 요청해보기

LLM 을 처음 시작한다면, LLM 에 몇가지 입력 (`Input`) 을 해보는 것이다. Prediction 에서 시도해볼 것은, `기업이 무엇을 하는지 예측해서, 기업의 이름을 생성` 하는 것을 시도해볼 것이다.

이를 위해서는, `LLM` Wrapper 패키지를 `import` 해야 한다.

In [7]:
from langchain.llms import OpenAI

`OpenAI` Class 를 초기화 할 때, `temperature` 파라메터를 설정할 수 있는데 (뭐 파라메터는 많겠지만, 여기에서는 `temperature` 파라메터만 다룬다.) 해당 파라메터를 높고 낮음의 따라서, 답변의 튀는 값 (`Random`) 을 높이고 낮출 수 있다고 한다.

뭐, 내가 틀린 이야기를 한 것일 수 있지만, 지금 상황에서는 그렇게 이해하고 있다. `temperature` 파라메터가 어떠한 역할을 하는지 궁금하다면, 다음의 문서를 읽어보길 바란다. 값은 항상 `0` 혹은 `1` 사이의 값이 되어야 한다고 하는데, `1` 을 `100%` 라고 생각하면 쉬울 것이다.

> It is always a number between 0 and 1.
>
> A temperature of 0 means the responses will be very straightforward, almost deterministic (meaning you almost always get the same response to a given prompt)
>
> A temperature of 1 means the responses can vary wildly.

How to use OpenAI model temperature?, https://gptforwork.com/guides/openai-gpt3-temperature

In [8]:
llm = OpenAI(temperature=0.9)

In [9]:
text = "알록달록한 양말을 만드는 회사를 위한 회사이름을 만들어 줘!"

In [10]:
print(llm(text))



1. Rainbow Sock Studio
2. Argyle Alley
3. Cheery Socks Co.
4. Fancy Feet
5. Jolly Toes
6. Colorful Footprints
7. Bright Strides
8. Tweedle Toes
9. Pompom Paws
10. Happy Hose Co.


일단은, 한국어로 질문을 했는데, 답변은 영어로 한다. 거 참 똑똑하다.

답변을 한국어로 하도록 설정해보자.

In [11]:
text = """
알록달록한 양말을 만드는 회사를 위한 회사이름을 만들어 줘!

답변은 한국어로 해줬으면 좋겠어!
"""

In [12]:
print(llm(text))


"레이니 색상 스토어" 또는 "색상 소외스토어"


파라메터에 한국어가 있는 줄 알았지만, 그냥 `답변을 한국어로 해줬으면 좋겠어!` 라고 이야기하니 한국어로 답변한다!

어쨌거나, 이게 `OpenAI` 패키지를 통한 맛보기라고 한다. 더 깊게 들어가고 싶다면, 다음의 문서를 참고하라고 한다.

* `LLM Getting Started Guide`: https://python.langchain.com/en/latest/modules/models/llms/getting_started.html

### LLM Prompt 전처리하기

위에서 본 것처럼, `LLM` 의 사용방법은 단순하지만, 문제는 `LLM` 에 요청하기 전에 약간의 전처리를 하고 싶을 때가 있을 것이다.

가령 예를 들어, 사용자가 입력한 문장 옆에 추가적인 Context 를 넣어서, `LLM` 이 맥락을 이해할 수 있게끔 말이다. (마치 위에서 본 것처럼 답변을 한국어로 해달라고 한 것처럼?)

이를 위해서는 `PromptTemplate` 를 사용해서 `Templating` 을 해볼 수 있다.

In [13]:
from langchain.prompts import PromptTemplate

In [14]:
prompt = PromptTemplate(
    input_variables=["language", "product"],
    template="What is a good name for a company that makes {product}?, please respond using {language}"
)

위의 코드를 통해서, Python 에서 함수 정의를 할 때 파라메터를 정의하듯이, `LLM` 을 호출 할 때 추가적인 파라메터를 정의하였다.

원래 코드는 `What is a good name for a company that makes {product}` 이지만, 우리는 특정 언어로 대답하도록, 해당 문장 뒤에 `please respond using {language}` 를 추가하였다.

정의한 템플릿을 통해서, Input 텍스트를 다음과 같이 생성할 수 있다.

In [15]:
input_text = prompt.format(product="Socks", language="Korean")

In [16]:
print(input_text)

What is a good name for a company that makes Socks?, please respond using Korean


그리고 이걸 위에서 사용했던 `llm` 을 통해서 생성하면 다음과 같다.

In [18]:
print(llm(input_text))



양말가게


`PromptTemplate` 을 사용하는 방법에 대해서는, 다음의 문서를 참고하면 된다.

https://python.langchain.com/en/latest/modules/prompts/chat_prompt_template.html

### Chain: LLM 과 다른 리소스를 연결해 보자!

`Chain` 을 통해서, 다른 리소스와 `LLM` 을 연결할 수 있다고 한다. 일단 우리는 위해서 정의했던 `LLM` 과 `PromptTemplate` 을 연결해 보자.

In [19]:
from langchain.chains import LLMChain

In [20]:
chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
english_response = chain.run(product="Rainbow", language="English")
korean_response = chain.run(product="Rainbow", language="Korean")

In [23]:
print(f"English Response: {english_response}")

English Response: 

Rainbow Creationz


In [24]:
print(f"한국어 응답: {korean_response}")

한국어 응답: 

레인보우메이커 (Reinbou Meikeo)


`LLMChain` 뿐만 아니라, 다른 Chain 함수를 통해서 여러가지의 리소스를 Chain 할 수 있다고 한다.
 
더 알고 싶다면, 다음의 문서를 읽어보길 바란다.

https://python.langchain.com/en/latest/modules/chains/getting_started.html